In [38]:
import matplotlib.pyplot as plt
from sklearn import datasets
import mpl_toolkits.mplot3d 
from sklearn.cluster import KMeans
from sklearn import metrics
import numpy as np
import pandas as pd
from cxplain.xkm import Xkm
from cxplain.tree import  DecisionTreeExplainer, RandomForestExplainer, ExKMCExplainer
from cxplain.shap import  ShapExplainer
from cxplain.gradient import GradientExplainer  
from cxplain.metrics import EuclideanMetric, Metric, ManhattenMetric
from cxplain.neon import NeonKMeansExplainer
from cxplain.errors import NonExistingRelevanceError

In [ ]:
dataset_index = {"iris": }
datasets = {"iris": }

First I only use the iris data set for evaluation

In [87]:
n_clusters = 4

In [88]:
iris = datasets.load_iris()
X = iris.data
y = iris.target
n_obs = X.shape[0]
n_features = X.shape[1]

In [89]:
# fit Kmeans
kmeans = KMeans(n_clusters=n_clusters, random_state=3).fit(X)
cluster_centers = kmeans.cluster_centers_
predictions = kmeans.predict(X)

In [90]:
# init and fit explainer
# list allexplainers
explainers = {"tree": DecisionTreeExplainer(data= X, cluster_predictions=predictions),
             "forest": RandomForestExplainer(data= X, cluster_predictions=predictions),
             "exkmc": ExKMCExplainer(X, kmeans, k=n_clusters, max_leaves=2*n_clusters),
             "gradient": GradientExplainer(X, cluster_centers, predictions, EuclideanMetric, enable_abs_calculation=False),
             "shap": ShapExplainer(data= X, cluster_predictions=predictions),
             "neon": NeonKMeansExplainer(cluster_centers=cluster_centers, data=X, predictions=predictions),
             "xkm": Xkm(data,  kmeans.cluster_centers_, "euclidean", predictions)}

# fit and explain all explainers

explanations = {explainer_name:explainer.fit_explain() for explainer_name, explainer  in explainers.items()}


In [91]:
feature_obs = np.array([0 for i in range(n_features)])
feature_obs.put(0, 0.2)
feature_obs

array([0, 0, 0, 0])

In [92]:
predictions[52]

0

In [93]:
X[52,:]

array([6.9, 3.1, 4.9, 1.5])

In [94]:
cluster_centers

array([[6.25714286, 2.86190476, 4.85      , 1.63333333],
       [5.006     , 3.428     , 1.462     , 0.246     ],
       [6.95      , 3.10666667, 5.86666667, 2.15333333],
       [5.53214286, 2.63571429, 3.96071429, 1.22857143]])

In [95]:
distances = [np.linalg.norm(X[52,:] - center) for center in cluster_centers]
distances

[0.7001659991058231, 4.133664234066429, 1.1678327505826052, 1.7442836667824653]

In [96]:
list(explanations["xkm"].pointwise_relevance.iloc[0, :])

[0.9096447061742748,
 0.9351574921337319,
 0.9988282455589736,
 0.9960078303263973]

In [97]:
# first calculate all ROC curves for individual observations
result_individual = {}
for explainer_name, explanation in explanations.items():
    # init curve_list
    curve_list = []
    for index_obs in range(n_obs):
        # init list curve_obs_i to all 1 (length = num_features)
        curve_obs = [1 for i in range(n_features)]
        # init array of feature observations, I use an array instead of a list, as it is easier  later on to calculate distances to cluster centers
        feature_obs = np.array([0.0 for i in range(n_features)])
        # get relevance scores for observation, for explainers with only global scores, these will be used for every observation
        try:
            relevance_scores = list(explanations[explainer_name].pointwise_relevance.iloc[index_obs, :])
        except NonExistingRelevanceError:
            relevance_scores = list(explanations[explainer_name].global_relevance)
        
        for feature_index in range(n_features):
            # get biggest score and column index (indicate which feature is meant) and pop from list
            index_biggest_score = relevance_scores.index(max(relevance_scores))                
            relevance_scores[index_biggest_score] = -100 # I set to large negative number as popping would ruin the index correspondence from relevance score to feature
            # get observation for this feature
            obs_biggest_score = X[index_obs, index_biggest_score]
            # get corresponding cluster index for this observation
            cluster_index = predictions[index_obs]
            # add observation for feature to feature observations list
            feature_obs.put(index_biggest_score, obs_biggest_score) # has to be at index of feature in training data, as otherwise distance calculation is wrong
            # impute other entries (length = num_features) --> TBD
            # calculate distance to cluster centers for feature observations list
            distances = [np.linalg.norm(feature_obs - center) for center in cluster_centers]
            # get nearest_cluster_index
            nearest_cluster_index = distances.index(min(distances))
            # check whether cluster_index == nearest_cluster_index
            # if yes: return curve_obs_i
            # if no: replace first entry of curve_obs_i ith 0 and repeat
            if cluster_index == nearest_cluster_index:
                break
            else:
                curve_obs[feature_index] = 0
            # if yes: return curve_obs_i
            # if no: replace first entry of curve_obs_i ith 0 and repeat
            
        curve_list.append(curve_obs)
        
    # add explainer entry to dict
    result_individual[explainer_name] = curve_list
      
# Now compute AUC
result_auc = {explainer_name: (1 /(n_obs*n_features)) * sum(map(sum, curves)) for explainer_name, curves in result_individual.items()}
# TBD

In [98]:
result_auc

{'tree': 0.66,
 'forest': 0.7050000000000001,
 'exkmc': 0.7183333333333334,
 'gradient': 0.6166666666666667,
 'shap': 0.6950000000000001,
 'neon': 0.6516666666666667,
 'xkm': 0.6516666666666667}

In [84]:
(1 /(n_obs*n_features)) * sum(map(sum, result_individual["tree"]))

0.7366666666666667

In [25]:
list(explanations["neon"].global_relevance)

[1.795856447363803, 1.7871515661301647, 1.7706217279596324, 1.7297476242693934]

In [30]:
feature_obs = np.array([0 for i in range(n_features)])

In [32]:
feature_obs[1] = 3
feature_obs

array([0, 3, 0, 0])

# How should the data structure look like in the end?
result_individual = {"explainer": curve_list}
curve_list = [curve_obs_i]
curve_obs_i = [indicator_feature_i]

result_auc = {"explainer": auc}
auc = (1/(num_obs * num_feat)) * sum(curve_list_agg)
curve_list_agg = sum(curve_list[curve_obs_i])